# Tool Use with LM Studio: Math Calculator

## Overview
This notebook demonstrates function/tool calling capabilities in LM Studio using a simple math calculator as an example.

## What is Tool/Function Calling?
Tool calling allows the model to request execution of specific functions to get information or perform actions. The model decides when to use a tool based on the user's query.

## Prerequisites
- LM Studio running with a model that supports function calling
- Python with `requests`, `pydantic`, and `json` libraries
- The `api_models.py` file in the same directory

## ⚠️ Compatibility Note
Function calling in LM Studio is **model-dependent** with ~60% compatibility. Models that typically support it:
- Mistral models with Instruct versions
- Some Llama models
- Models specifically fine-tuned for function calling

**Test Model**: `mistralai/magistral-small-2509`

## 1. Setup and Imports

Import necessary libraries and configure the connection to LM Studio.

In [1]:
# Standard library imports
import json
import requests
from typing import Dict, Any, Optional

# Import Pydantic models from our custom module
from api_models import (
    ChatCompletion,
    ChatMessage,
    Tool,
    Function,
    create_chat_message,
    create_chat_completion_request
)

In [2]:
# Configure LM Studio connection
BASE_URL = "http://169.254.83.107:1234/v1"
CHAT_ENDPOINT = f"{BASE_URL}/chat/completions"

# Model to use (must support function calling)
MODEL = "mistralai/magistral-small-2509"

print(f"🔧 Configuration:")
print(f"  • Base URL: {BASE_URL}")
print(f"  • Model: {MODEL}")

🔧 Configuration:
  • Base URL: http://169.254.83.107:1234/v1
  • Model: mistralai/magistral-small-2509


## 2. Define the Math Calculator Function

Create the actual Python function that will perform calculations.

In [3]:
def math_calculator(arg1: float, arg2: float, operation: str) -> Dict[str, Any]:
    """
    Perform basic mathematical operations on two numbers.
    
    Args:
        arg1: First number
        arg2: Second number
        operation: Mathematical operation (add, subtract, multiply, divide)
    
    Returns:
        Dictionary with the result or error message
    """
    try:
        if operation == "add":
            result = arg1 + arg2
            expression = f"{arg1} + {arg2}"
        elif operation == "subtract":
            result = arg1 - arg2
            expression = f"{arg1} - {arg2}"
        elif operation == "multiply":
            result = arg1 * arg2
            expression = f"{arg1} × {arg2}"
        elif operation == "divide":
            if arg2 == 0:
                return {
                    "success": False,
                    "error": "Division by zero is not allowed"
                }
            result = arg1 / arg2
            expression = f"{arg1} ÷ {arg2}"
        else:
            return {
                "success": False,
                "error": f"Unknown operation: {operation}"
            }
        
        return {
            "success": True,
            "expression": expression,
            "result": result
        }
    
    except Exception as e:
        return {
            "success": False,
            "error": str(e)
        }

# Test the function
print("🧪 Testing math_calculator:")
print(f"  • 10 + 5 = {math_calculator(10, 5, 'add')}")
print(f"  • 20 ÷ 4 = {math_calculator(20, 4, 'divide')}")
print(f"  • 5 ÷ 0 = {math_calculator(5, 0, 'divide')}")

🧪 Testing math_calculator:
  • 10 + 5 = {'success': True, 'expression': '10 + 5', 'result': 15}
  • 20 ÷ 4 = {'success': True, 'expression': '20 ÷ 4', 'result': 5.0}
  • 5 ÷ 0 = {'success': False, 'error': 'Division by zero is not allowed'}


## 3. Define the Tool Schema

Create the OpenAI-compatible tool definition that describes our function to the model.

In [5]:
# Define the tool schema for the model
math_tool_schema = {
    "type": "function",
    "function": {
        "name": "math_calculator",
        "description": "Perform mathematical calculations with two numbers",
        "parameters": {
            "type": "object",
            "properties": {
                "arg1": {
                    "type": "number",
                    "description": "The first number in the calculation"
                },
                "arg2": {
                    "type": "number",
                    "description": "The second number in the calculation"
                },
                "operation": {
                    "type": "string",
                    "enum": ["add", "subtract", "multiply", "divide"],
                    "description": "The mathematical operation to perform"
                }
            },
            "required": ["arg1", "arg2", "operation"]
        }
    }
}

# List of all available tools
tools = [math_tool_schema]

print("📋 Tool Schema defined:")
print(json.dumps(math_tool_schema, indent=2))

📋 Tool Schema defined:
{
  "type": "function",
  "function": {
    "name": "math_calculator",
    "description": "Perform mathematical calculations with two numbers",
    "parameters": {
      "type": "object",
      "properties": {
        "arg1": {
          "type": "number",
          "description": "The first number in the calculation"
        },
        "arg2": {
          "type": "number",
          "description": "The second number in the calculation"
        },
        "operation": {
          "type": "string",
          "enum": [
            "add",
            "subtract",
            "multiply",
            "divide"
          ],
          "description": "The mathematical operation to perform"
        }
      },
      "required": [
        "arg1",
        "arg2",
        "operation"
      ]
    }
  }
}


## 4. Helper Function for Tool Calling Workflow

Create a helper function that handles the complete tool calling workflow.

In [ ]:
def execute_tool_call(tool_name: str, arguments: Dict[str, Any]) -> str:
    """
    Execute the requested tool with provided arguments.
    
    Args:
        tool_name: Name of the tool to execute
        arguments: Arguments for the tool
    
    Returns:
        JSON string with the result
    """
    if tool_name == "math_calculator":
        result = math_calculator(
            arg1=arguments.get("arg1"),
            arg2=arguments.get("arg2"),
            operation=arguments.get("operation")
        )
        return json.dumps(result)
    else:
        return json.dumps({"error": f"Unknown tool: {tool_name}"})


def chat_with_tools(user_query: str, show_details: bool = True) -> str:
    """
    Send a query to the model with tool support.
    
    Args:
        user_query: The user's question
        show_details: Whether to print detailed information
    
    Returns:
        The model's final response
    """
    # Step 1: Initial conversation
    messages = [
        create_chat_message("system", "You are a helpful assistant with access to a math calculator tool. Use it when needed to perform calculations."),
        create_chat_message("user", user_query)
    ]
    
    # Step 2: Send request with tools
    request_data = {
        "model": MODEL,
        "messages": [msg.model_dump(exclude_none=True) for msg in messages],
        "tools": tools,
        "tool_choice": "auto",  # Let the model decide when to use tools
        "temperature": 0.1  # Low temperature for consistent math
    }
    
    if show_details:
        print(f"\n📤 Sending query: {user_query}")
    
    response = requests.post(
        CHAT_ENDPOINT,
        json=request_data,
        timeout=30
    )
    response.raise_for_status()
    
    # Step 3: Parse initial response
    completion = ChatCompletion.model_validate(response.json())
    assistant_message = completion.choices[0].message
    
    # Step 4: Check if the model wants to use a tool
    if assistant_message.tool_calls:
        if show_details:
            print("\n🔧 Model requested tool use:")
        
        # Add assistant's message to conversation
        messages.append(assistant_message)
        
        # Process each tool call
        for tool_call in assistant_message.tool_calls:
            function_name = tool_call.function.name
            function_args = json.loads(tool_call.function.arguments)
            
            if show_details:
                print(f"  • Function: {function_name}")
                print(f"  • Arguments: {function_args}")
            
            # Execute the tool
            tool_result = execute_tool_call(function_name, function_args)
            
            if show_details:
                print(f"  • Result: {tool_result}")
            
            # Add tool result to conversation
            tool_message = create_chat_message(
                "tool",
                tool_result
            )
            tool_message.tool_call_id = tool_call.id
            messages.append(tool_message)
        
        # Step 5: Get final response from model
        final_request = {
            "model": MODEL,
            "messages": [msg.model_dump(exclude_none=True) for msg in messages],
            "temperature": 0.7
        }
        
        final_response = requests.post(
            CHAT_ENDPOINT,
            json=final_request,
            timeout=30
        )
        final_response.raise_for_status()
        
        final_completion = ChatCompletion.model_validate(final_response.json())
        return final_completion.choices[0].message.content
    
    else:
        # Model didn't use tools, return direct response
        if show_details:
            print("\n💬 Model responded without using tools")
        return assistant_message.content

print("✅ Helper functions defined")

✅ Helper functions defined


## 5. Example 1: Simple Addition

Let's start with a simple calculation request.

In [8]:
# Simple addition example
query = "What is 42 plus 17?"
result = chat_with_tools(query)

print("\n🤖 Model's Final Response:")
print("-" * 40)
print(result)


📤 Sending query: What is 42 plus 17?

🔧 Model requested tool use:
  • Function: math_calculator
  • Arguments: {'arg1': 42, 'arg2': 17, 'operation': 'add'}
  • Result: {"success": true, "expression": "42 + 17", "result": 59}

🤖 Model's Final Response:
----------------------------------------
The result of 42 plus 17 is **59**.


## 6. Example 2: Division

Test division operation with decimal results.

In [9]:
# Division example
query = "Can you divide 100 by 7 for me?"
result = chat_with_tools(query)

print("\n🤖 Model's Final Response:")
print("-" * 40)
print(result)


📤 Sending query: Can you divide 100 by 7 for me?

🔧 Model requested tool use:
  • Function: math_calculator
  • Arguments: {'arg1': 100, 'arg2': 7, 'operation': 'divide'}
  • Result: {"success": true, "expression": "100 \u00f7 7", "result": 14.285714285714286}

🤖 Model's Final Response:
----------------------------------------
The result of dividing 100 by 7 is approximately 14.285714285714286.


## 7. Example 3: Multiple Calculations

Test with a query that might require multiple calculations.

In [10]:
# Complex query
query = "I have 150 apples. I give away 37 apples. How many do I have left?"
result = chat_with_tools(query)

print("\n🤖 Model's Final Response:")
print("-" * 40)
print(result)


📤 Sending query: I have 150 apples. I give away 37 apples. How many do I have left?

🔧 Model requested tool use:
  • Function: math_calculator
  • Arguments: {'arg1': 150, 'arg2': 37, 'operation': 'subtract'}
  • Result: {"success": true, "expression": "150 - 37", "result": 113}

🤖 Model's Final Response:
----------------------------------------
You started with 150 apples and gave away 37. You have 113 apples left.


## 8. Example 4: Error Handling

Test division by zero to see error handling.

In [11]:
# Division by zero
query = "What is 10 divided by 0?"
result = chat_with_tools(query)

print("\n🤖 Model's Final Response:")
print("-" * 40)
print(result)


📤 Sending query: What is 10 divided by 0?

💬 Model responded without using tools

🤖 Model's Final Response:
----------------------------------------
The result of dividing any number by zero is undefined in mathematics. This is because there is no finite number that can be multiplied by zero to produce a non-zero number. Therefore, the expression \( \frac{10}{0} \) does not have a defined value.


## 9. Batch Processing Examples

Process multiple calculations to see patterns.

In [12]:
# Multiple test cases
test_queries = [
    "What is 25 times 4?",
    "Calculate 1000 minus 567",
    "What's the result of 12.5 multiplied by 8?",
    "Divide 144 by 12"
]

print("🔄 Running batch calculations:\n")

for query in test_queries:
    print(f"Question: {query}")
    result = chat_with_tools(query, show_details=False)
    print(f"Answer: {result}\n")
    print("-" * 50 + "\n")

🔄 Running batch calculations:

Question: What is 25 times 4?
Answer: The result of multiplying 25 by 4 is 100.

--------------------------------------------------

Question: Calculate 1000 minus 567
Answer: The result of 1000 minus 567 is 433.

--------------------------------------------------

Question: What's the result of 12.5 multiplied by 8?
Answer: The result of 12.5 multiplied by 8 is 100.

--------------------------------------------------

Question: Divide 144 by 12
Answer: The result of dividing 144 by 12 is 12.

--------------------------------------------------



## 10. Testing Without Tool Support

See what happens when we ask a math question without providing tools.

In [13]:
# Test without tools to see the difference
def chat_without_tools(query: str) -> str:
    """Send a query without tool support for comparison."""
    messages = [
        create_chat_message("system", "You are a helpful assistant."),
        create_chat_message("user", query)
    ]
    
    request_data = {
        "model": MODEL,
        "messages": [msg.model_dump(exclude_none=True) for msg in messages],
        "temperature": 0.7
    }
    
    response = requests.post(
        CHAT_ENDPOINT,
        json=request_data,
        timeout=30
    )
    response.raise_for_status()
    
    completion = ChatCompletion.model_validate(response.json())
    return completion.choices[0].message.content

# Compare responses with and without tools
query = "What is 357 multiplied by 29?"

print("🔍 Comparison: With Tools vs Without Tools\n")
print(f"Query: {query}\n")

print("WITH TOOLS:")
with_tools = chat_with_tools(query, show_details=False)
print(f"Response: {with_tools}\n")

print("WITHOUT TOOLS:")
without_tools = chat_without_tools(query)
print(f"Response: {without_tools}")

🔍 Comparison: With Tools vs Without Tools

Query: What is 357 multiplied by 29?

WITH TOOLS:
Response: The result of 357 multiplied by 29 is 10,353.

WITHOUT TOOLS:
Response: To calculate \( 357 \times 29 \), we can use the distributive property of multiplication over addition, which allows us to break down the problem into simpler parts.

First, let's express 29 as \( 30 - 1 \):

\[
357 \times 29 = 357 \times (30 - 1)
\]

Now, distribute the multiplication:

\[
357 \times 30 - 357 \times 1
\]

Calculate each part separately:

1. \( 357 \times 30 \):

\[
357 \times 30 = (300 + 50 + 7) \times 30 = 300 \times 30 + 50 \times 30 + 7 \times 30
\]
\[
= 9000 + 1500 + 210 = 10,710
\]

2. \( 357 \times 1 \):

\[
357 \times 1 = 357
\]

Now, subtract the two results:

\[
10,710 - 357 = 10,353
\]

So,

\[
357 \times 29 = 10,353
\]


## Summary

This notebook demonstrated tool/function calling with LM Studio:

### What We Learned

1. **Tool Definition**: Created a math calculator function with clear parameters
2. **Schema Creation**: Defined OpenAI-compatible tool schema
3. **Workflow Implementation**: Built complete tool calling workflow
4. **Error Handling**: Handled edge cases like division by zero
5. **Comparison**: Showed difference between responses with and without tools

### Key Takeaways

- **Model Dependency**: Function calling works differently across models
- **Structured Flow**: Tool calls follow a specific request-execute-respond pattern
- **Type Safety**: Pydantic models help validate tool calls and responses
- **Error Handling**: Always handle potential errors in tool execution
- **Accuracy**: Tools provide precise calculations vs. model approximations

### Extending This Example

You can create more complex tools:
- **Database queries**: Fetch real-time data
- **API calls**: Integrate external services
- **File operations**: Read/write local files
- **System commands**: Execute system operations
- **Multi-step workflows**: Chain multiple tools together

### Troubleshooting

If function calling doesn't work:
1. **Check model compatibility**: Not all models support function calling
2. **Verify schema**: Ensure tool schema matches OpenAI format
3. **Test with OpenAI SDK**: Try using official OpenAI Python client
4. **Check LM Studio logs**: Look for error messages in server logs
5. **Update LM Studio**: Ensure you have the latest version